<a href="https://colab.research.google.com/github/yo20mom/CUK_Learning/blob/master/KerasDeepLearning/KERAS_REPORT_CNN_%EB%A7%88%EC%82%AC%EC%A7%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, utils, callbacks, datasets, layers, applications, optimizers
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Activation, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras import backend as K

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
NUM_EPOCHS = 33
INIT_LEARNING_RATE= 1e-2
BATCH_SIZE = 64

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
def my_model(filters=[32, 64], kernel_size=(3, 3), pool_size=(2, 2), act='relu'):
  width, height, depth, classes = 28, 28, 1, 10
  
  model = Sequential()
  input_shape = (height, width, depth)
  chan_dim = -1

  for i, x in enumerate(filters):
    if i == 0:
      model.add(Conv2D(filters=x, 
                       kernel_size=kernel_size, 
                       padding='same',
                       input_shape=input_shape))
    else:
      model.add(Conv2D(filters=x, kernel_size=kernel_size, padding='same'))

    model.add(BatchNormalization(axis=chan_dim))
    model.add(Activation(act))
    model.add(Conv2D(filters=x, kernel_size=kernel_size, padding='same'))
    model.add(BatchNormalization(axis=chan_dim))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.3))

  model.add(Flatten())
  
  model.add(Dense(512))
  model.add(BatchNormalization())
  model.add(Activation(act))
  model.add(Dropout(0.5))

  model.add(Dense(512))
  model.add(BatchNormalization())
  model.add(Activation(act))
  model.add(Dropout(0.5))

  model.add(Dense(classes))
  model.add(Activation('softmax'))

  return model

In [ ]:
x_train.shape, x_test.shape

In [ ]:
x_train =x_train.reshape(x_train.shape[0], 28, 28,1)
x_test = x_test.reshape(x_test.shape[0],28 ,28,1)

In [ ]:
x_train.shape, x_test.shape

#값 정규화

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [ ]:
x_train[83][11]

In [ ]:
label_names = ["top", "trouser", "pullover", "dress", "coat", "sandal", "shirt", "sneaker", "bag", "ankle boot"]
def get_answer(num):
    return label_names[num]

In [ ]:
row = 77
#print(get_answer(y_train[row]))
plt.imshow(x_train[row].reshape(28, 28))

# 원핫 인코딩으로 변경

In [ ]:
classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes=classes)
y_test = to_categorical(y_test, num_classes=classes)

In [ ]:
# 모델 저장 기준이 되는 콜백 import
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# 체크할 포인트
check_point = ModelCheckpoint('best_model.h5', 
                              monitor='val_accuracy',
                              verbose=1,
                              save_best_only=True)

# 이른 스탑을 위한 클래스
early_stopping = EarlyStopping(monitor='val_accruacy',
                               min_delta=0,
                               patience=7,
                               verbose=1)

# 모델 컴파일 및 학습


In [ ]:
bat_size = 128
epochs = 33
for lr, bs, ep in zip([0.001], [bat_size] * 1, [epochs] * 1):
  optimizer = Adam(learning_rate=lr, 
                  decay=lr / ep, 
                  beta_1=0.9,
                  beta_2=0.999)


  model = my_model()

  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  history = model.fit(x_train,
                      y_train,
                      batch_size=bs,
                      validation_split=0.3,
                      validation_data=(x_test, y_test),
                      epochs=ep,
                      verbose=2,
                      callbacks=[check_point, early_stopping])

In [ ]:
loaded_model = load_model('best_model.h5')

prob_pred = loaded_model.predict(x_test)
pred = loaded_model.evaluate(x_test, y_test)
print(pred)

prob_label = prob_pred.argmax(axis=-1)

np.savetxt('y_pred.csv', prob_label, fmt='%d')